In [1]:
%load_ext autoreload
%autoreload 2

import datetime
import multiprocessing
import multiprocess
import feedparser as fp
import newspaper
from newspaper import news_pool
import helpers
from newspaper import Source, Article
import pandas as pd

## 1.
This is downloading newspapers as one would conventionally using the $newspaper3k$ package 

In [2]:
%%time
# Do conventional newspaper3k build...
cnn_3k_paper = newspaper.build('https://www.cnn.com/', memoize_articles=False)
rt_3k_paper = newspaper.build('https://www.rt.com/', memoize_articles=False)
bbc_3k_paper = newspaper.build('https://www.bbc.com/news', memoize_articles=False)
fox_3k_paper = newspaper.build('https://www.foxnews.com/', memoize_articles=False)
msnbc_3k_paper = newspaper.build('https://www.msnbc.com/', memoize_articles=False)

CPU times: user 7.06 s, sys: 835 ms, total: 7.9 s
Wall time: 27.4 s


In [3]:
%%time
# ... and multithread download:
papers = [cnn_3k_paper, 
          rt_3k_paper,
          bbc_3k_paper,
          fox_3k_paper,
          msnbc_3k_paper]
news_pool.set(papers, threads_per_source=2) # (5*2) = 10 threads total
news_pool.join()

CPU times: user 1min 8s, sys: 9.96 s, total: 1min 18s
Wall time: 9min 37s


In [4]:
%%time
# Process the downloaded texts:
for paper in papers:
    paper.parse_articles()

CPU times: user 7min 30s, sys: 16 s, total: 7min 46s
Wall time: 8min 15s


In [5]:
filepath = 'data_extra/conventional_data_pulls/'

In [6]:
%%time
for paper in papers:
    helpers.write_out_articles(newspaper_obj=paper,
                               filepath=filepath, 
                               writeout_type='conventional')

CPU times: user 27.7 s, sys: 4.97 s, total: 32.7 s
Wall time: 34.6 s


Check for files:

In [11]:
df = pd.read_csv(filepath + '2019_3_18_bbc.csv')

In [12]:
df.text

0      Image copyright @PolitieUtrecht / Twitter Imag...
1      Media playback is unsupported on your device M...
2      Media playback is unsupported on your device M...
3      Media playback is unsupported on your device M...
4      Image copyright Nebraska DOT\n\nThawing snowpa...
5      Image copyright Pipa Image caption Armando, pi...
6      Media playback is unsupported on your device M...
7      Image copyright AFP/Getty Images Image caption...
8      Image copyright Facebook Image caption Images ...
9      Image copyright AFP Image caption Koffi Olomid...
10     Image copyright Getty Images Image caption Bet...
11     Image copyright Getty Images Image caption Tho...
12     Image copyright Getty Images Image caption Tho...
13     Image copyright Megan Janetsky Image caption E...
14     Image copyright Mansi Thapliyal Image caption ...
15     Image copyright Getty Images\n\nIt's no secret...
16     Media playback is unsupported on your device M...
17     Image copyright Getty Im